In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

df = yf.download(tickers='AAPL', start='2010-01-01', end='2023-12-31', auto_adjust=False, progress=False)
df.reset_index(inplace=True)
if 'Date' in df.index.names:
    df = df.reset_index('Date')
    df.drop('Date', axis=1, inplace=True)
print("Data loaded:", df.shape)

Data loaded: (3522, 7)


In [3]:
training_size = int(len(df) * 0.8)
data_training = pd.DataFrame(df['Close'][0:training_size])

scaler = MinMaxScaler(feature_range=(0,1))
data_training_scaled = scaler.fit_transform(data_training)

x_train = []
y_train = []

for i in range(100, len(data_training_scaled)):
    x_train.append(data_training_scaled[i-100:i, 0])
    y_train.append(data_training_scaled[i, 0])

x_train = np.array(x_train)
y_train = np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

print("Training data prepared:", x_train.shape, y_train.shape)

Training data prepared: (2717, 100, 1) (2717,)


In [5]:
# Build model
model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(100, 1)),
    Dropout(0.2),
    LSTM(units=60, return_sequences=True),
    Dropout(0.3),
    LSTM(units=80, return_sequences=True),
    Dropout(0.4),
    LSTM(units=120),
    Dropout(0.5),
    Dense(units=1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
print("Model built successfully")

Model built successfully


In [6]:
# Train and save
model.fit(x_train, y_train, epochs=10, batch_size=32, verbose=1)  # Reduced epochs for testing
model.save('keras_model.h5')
print("Model saved successfully!")


Epoch 1/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 11s 83ms/step - loss: 0.0140
Epoch 2/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 82ms/step - loss: 0.0024
Epoch 3/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 82ms/step - loss: 0.0023
Epoch 4/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 79ms/step - loss: 0.0020
Epoch 5/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 80ms/step - loss: 0.0015
Epoch 6/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 80ms/step - loss: 0.0018
Epoch 7/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 82ms/step - loss: 0.0019
Epoch 8/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 80ms/step - loss: 0.0012
Epoch 9/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 80ms/step - loss: 0.0016
Epoch 10/10
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 80ms/step - loss: 0.0014


Model saved successfully!
